In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
load_dotenv()

In [ ]:
model = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
loader = PyPDFLoader('intro-to-ml.pdf')
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [ ]:
len(chunks)

In [ ]:
embeddings = OpenAIEmbeddings(model = "text-embeddings-3-small")
vector_store = FAISS.from_documents(chunks , embeddings)

In [ ]:
vector_store

In [ ]:
retriever = vector_store.as_retriever(search_type='similarity',search_kwrgs={'k':4})


In [ ]:
@tool
def rag_tool(query):
   """
    Retrieve relevant information from the pdf document.
    Use this tool when the user asks factual / conceptual questions
    that might be answered from the stored documents.
    """
   result = retriever.invoke(query)

   context = [doc.page_content for doc in result]
   metadata = [doc.metadata for doc in result]

   return {
      'query': query,
      'context': context,
      "metadata": metadata
   }

In [ ]:
tools = [rag_tool]

llm_with_tools = model.bind_tools(tools)


In [ ]:
class ChatState(TypedDict):
  messages : Annotated[list[BaseMessage],add_messages]

In [ ]:
def chat_node(state: ChatState):

  messages = state['messages']

  response = llm_with_tools.invoke(messages)

  return {"messages": [response]}

In [ ]:
tool_node = ToolNode(tools)

In [ ]:
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)
graph.add_node('tools', tool_node)

graph.add_edge(START, 'chat_node')
graph.add_conditional_edges('chat_node', tools_condition)
graph.add_edge('tools', 'chat_node')

chatbot = graph.compile()

In [ ]:
chatbot

In [ ]:
result = chatbot.invoke(
    {
        "messages": [
            HumanMessage(
                content=(
                    "Using the pdf notes, explain how to find the ideal value of K in KNN"
                )
            )
        ]
    }
)

In [ ]:

print(result['messages'][-1].content)